In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
sn.set_style(style="darkgrid")

# **EDA**

In [ ]:
df=pd.read_csv("/kaggle/input/car-price-estimation/datasets_33080_43333_car data.csv")

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:

for i in list(df.columns)[2:5]:   
    
    sn.barplot(x="Transmission",y=i,data=df,palette="pastel")
    plt.show()

In [ ]:
for i in list(df.columns)[2:5]:
    
    sn.barplot(x="Transmission",y=i,data=df,hue="Fuel_Type",palette="cool")
    plt.show()

In [ ]:

sn.boxplot(df.Transmission,df.Selling_Price,palette="cool")
plt.show()

In [ ]:

sn.barplot(x="Transmission",y="Selling_Price",data=df,hue="Owner")
plt.show()

In [ ]:
sn.heatmap(df.corr(),annot=True)
plt.show()

# **Feature enginiering**

In [ ]:
manual=df[(df.Transmission=="Manual")]

In [ ]:
plt.subplots(1,1,figsize=(12,12))
sn.histplot(manual.Selling_Price,kde=True)
plt.show()

In [ ]:
lower_manual=manual.Selling_Price.quantile(0.30)
upper_manual=manual.Selling_Price.quantile(0.977)

In [ ]:
automatic=df[df.Transmission == "Automatic"]

In [ ]:
plt.subplots(1,1,figsize=(12,12))
sn.histplot(automatic.Selling_Price,kde=True)
plt.show()

In [ ]:
lower_automatic=automatic.Selling_Price.quantile(0.25)
upper_automatic=automatic.Selling_Price.quantile(0.999)

In [ ]:
def eliminate_out(dataframe,lower,upper):
    
    return dataframe[(dataframe.Selling_Price > lower) & (dataframe.Selling_Price < upper)]

In [ ]:
manual_not_out=eliminate_out(manual,lower_manual,upper_manual)
automatic_not_out=eliminate_out(automatic,lower_automatic,upper_automatic)

In [ ]:
df_not_out=pd.concat([automatic_not_out,manual_not_out])

In [ ]:
df_not_out

In [ ]:
sn.boxplot(df_not_out.Transmission,df_not_out.Selling_Price,palette="cool")
plt.show()

In [ ]:
df_not_out.columns

In [ ]:
data=df_not_out[['Present_Price','Transmission','Fuel_Type','Seller_Type',"Kms_Driven"]]
target=np.array(df_not_out.Selling_Price).reshape(-1,1)

In [ ]:
data=pd.get_dummies(data)

In [ ]:
data

In [ ]:
from sklearn.preprocessing import StandardScaler,PolynomialFeatures

In [ ]:
scala=StandardScaler()

In [ ]:
data_scala=scala.fit_transform(data)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(data_scala,
                                               target,
                                               test_size=0.2,
                                               random_state=0)

In [ ]:
X_test.shape,Y_test.shape

# **Linear Regression**

In [ ]:
from sklearn import linear_model

In [ ]:
reg_lineal=linear_model.LinearRegression()

In [ ]:
reg_lineal.fit(X_train,Y_train)

reg_lineal.score(X_test,Y_test)          

In [ ]:
reg_lineal.score(X_train,Y_train)

# **Polynomial Regression**

In [ ]:
poli=PolynomialFeatures(degree=2)

In [ ]:
def poli_tranform(x):
    
    return poli.fit_transform(x)

In [ ]:
X_train_poli=poli_tranform(X_train)
X_test_poli=poli_tranform(X_test)

In [ ]:
poli.fit(X_train_poli,Y_train)

In [ ]:
poli_reg=linear_model.LinearRegression()

In [ ]:
poli_reg.fit(X_train_poli,Y_train)

In [ ]:
poli_reg.score(X_test_poli,Y_test)

In [ ]:
poli_reg.score(X_train_poli,Y_train)

# **XGBOOST**

In [ ]:
from xgboost import XGBRegressor

xgb=XGBRegressor(max_depth=1)

xgb.fit(X_train,Y_train)

In [ ]:
xgb.score(X_test,Y_test)

In [ ]:
xgb.score(X_train,Y_train)

# **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf=RandomForestRegressor()

rf.fit(X_train,Y_train)

rf.score(X_test,Y_test)

In [ ]:
rf.score(X_train,Y_train)

# ****Metrics****

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
def MAE(y_true,y_pred):
    
    return mean_absolute_error(y_true,y_pred)


In [ ]:
rf_predict=rf.predict(X_test)
lineal_pred=reg_lineal.predict(X_test)

poli_pred=poli_reg.predict(X_test_poli)
xgb_pred=xgb.predict(X_test)

In [ ]:
print("MAE Random Forest", MAE(Y_test,rf_predict))

In [ ]:
print("XGBOOST", MAE(Y_test,xgb_pred))

In [ ]:
print("Linear Regression", MAE(Y_test,lineal_pred))

In [ ]:
print("Polynomial Regression", MAE(Y_test,poli_pred))

# **Conclution**



Linear, polynomial, and XGBOOST regression all perform well with a random forest exception that over-fits the training data.
The polynomial regression of degree 2 was superior to the XGBOOST but with the difference that it required more preprocessing, however the latter without doing much tuning to the data showed an excellent performance.

In [ ]:
df_test=pd.DataFrame({"True_Values": Y_test.flatten(), 
                      "Predict_values": poli_pred.flatten()})

In [ ]:
df_test

The model prediction is very close to the original results.

In [ ]:
def predict_vs_ytest(ax):
    
    return df_test.plot.scatter(x="True_Values",y="Predict_values",ax=ax,c="c",figsize=(12,8))

In [ ]:
ax=df_test.plot.scatter(x="Predict_values",y="True_Values",c="b")

plt.title("True values vs Predict values")

predict_vs_ytest(ax)
plt.legend(["True values","Predict values"])
plt.show()